In [1]:
import numpy as np
import scipy.signal
import tensorflow as tf
from pywt import wavedec
import pandas as pd
import wfdb
from Pan_Tompkins import Pan_Tompkins_QRS
from Heart_Rate import HeartRate
import cx_Oracle
from datetime import datetime

# Oracle DB 연결 설정
connection = cx_Oracle.connect('CGI_24S_IOT3_1', 'smhrd1', 'project-db-cgi.smhrd.com:1524/xe')
cursor = connection.cursor()

model = tf.keras.models.load_model('model0812_6s.keras')
sampling_rate = 360.0

def extract_features(signal):
    signal = scipy.signal.decimate(signal, 2)  # 다운 샘플링
    features = wavedec(signal, 'db6', level=4)  # 웨이브렛 변환

    padded_all = []
    for f in features:
        padded_feature = np.pad(f, (0, len(features[-1]) - len(f)), 'constant')
        padded_all.append(padded_feature)
    
    K = np.swapaxes(np.array(padded_all), 0, 1)
    F = np.expand_dims(K, axis=0)  # 배치 차원 추가
    return F

# RR 간격 및 심박수 계산 함수
def calculate_rr_intervals(signals, sampling_rate):
    ecg_df = pd.DataFrame({'TimeStamp': np.arange(len(signals)), 'ecg': signals})
    QRS_detector = Pan_Tompkins_QRS(fs=sampling_rate)
    mwin, bpass = QRS_detector.solve(ecg_df)

    hr = HeartRate(signals, sampling_rate, mwin, bpass)
    rpeaks = hr.find_r_peaks()

    if len(rpeaks) < 2:
        print("Insufficient R-peaks detected, unable to calculate intervals.")
        return 0.0, 0.0, 0.0, 0.0, 0.0

    rpeaks = np.array(rpeaks)
    rr_intervals = np.diff(rpeaks) / sampling_rate
    bpm = (60.0 / rr_intervals.mean())
    rr_min = rr_intervals.min() * 1000
    rr_max = rr_intervals.max() * 1000
    rr_avg = rr_intervals.mean() * 1000
    rr_std = rr_intervals.std() * 1000

    return bpm, rr_min, rr_max, rr_avg, rr_std

# 데이터 처리
path = '106'
record = wfdb.rdrecord(path)
signal = record.p_signal[:, 0]  
segment = signal[34560:36720]

test_features = extract_features(segment)
predictions = model.predict(test_features)[0]

class_labels = ['/', 'L', 'N', 'R', 'V']
analysis_result = ", ".join([f"{label}: {prob:.4f}" for label, prob in zip(class_labels, predictions)])

bpm, rr_min, rr_max, rr_avg, rr_std = calculate_rr_intervals(segment.flatten(), sampling_rate)

segment_blob = segment.tobytes()

print(analysis_result)
print(bpm, rr_min, rr_max, rr_avg, rr_std)

# 현재 시각 가져오기
created_at = datetime.now()

# tb_analysis 테이블에 데이터 삽입
sql = """
    INSERT INTO tb_analysis 
    (ID, BP_AVG, CREATED_AT, ANALISYS_RESULT, ANALISYS_ETC, ECG, RR_MIN, RR_MAX, RR_AVG, RR_STD, MODIFIED_AT) 
    VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11)
"""

# 데이터 준비
data = (
    '202502@jnu.ac.kr',  # ID 값
    bpm,  # BP_AVG
    created_at,  # CREATED_AT
    analysis_result,  # ANALISYS_RESULT
    'Test ECG Analysis',  # ANALISYS_ETC
    segment_blob,  # ECG
    rr_min,  # RR_MIN
    rr_max,  # RR_MAX
    rr_avg,  # RR_AVG
    rr_std,  # RR_STD
    created_at  # MODIFIED_AT
)
# SQL 실행
cursor.execute(sql, data)
connection.commit()

# DB 연결 종료
cursor.close()
connection.close()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step
/: 0.0000, L: 0.0000, N: 0.0000, R: 0.0000, V: 1.0000
90.50279329608937 0.0 1297.2222222222222 662.962962962963 444.90909353362997
